In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [ ]:
Target_dict = pd.read_pickle('../preprocessed_data/Target_dict.pkl')
herb_dict = pd.read_pickle('../preprocessed_data/Herb_dict.pkl')

In [ ]:
Herb_Target = pd.read_csv('../preprocessed_data/Preprocessed_Herb-Target.csv', index_col=None)
print(len(Herb_Target))
Herb_Target['Target'] = Herb_Target['Gene symbol'].map(Target_dict).dropna()
Herb_Target['Herb'] = Herb_Target['Herb'].map(herb_dict).dropna()
print(len(Herb_Target))

Herb_Target = Herb_Target[['Herb', 'Target', 'P_value', 'Inferred evidence score', 'FDR(BH)', 'FDR(Bonferroni)']]

In [ ]:
Herb_Target_P = Herb_Target[(Herb_Target['P_value'] <= 0.05) & (Herb_Target['Inferred evidence score'] >= 0.250)]
print(len(Herb_Target_P))

Herb_Target_N = Herb_Target[(Herb_Target['P_value'] <= 0.05) & (Herb_Target['Inferred evidence score'] <= 0.200)]
print(len(Herb_Target_N))

In [ ]:
to_drop = []

for i in range(698):
    Herb_Target_P_item = Herb_Target_P[Herb_Target_P['Herb'] == i]
    Herb_Target_N_item = Herb_Target_N[Herb_Target_N['Herb'] == i]

    if Herb_Target_P_item.shape[0] < 2:
        to_drop.append(i)

    if Herb_Target_N_item.shape[0] < 2:
        to_drop.append(i)

to_drop = list(set(to_drop))

Herb_Target_P = Herb_Target_P[~Herb_Target_P['Herb'].isin(to_drop)]
Herb_Target_N = Herb_Target_N[~Herb_Target_N['Herb'].isin(to_drop)]

print(len(Herb_Target_P))
print(len(Herb_Target_N))

In [ ]:
for random_seed in [10, 20, 30, 40, 50]:

    df_final = pd.DataFrame(columns=['herb', 'positive_target', 'negative_target', 'positive_score', 'negative_score'])

    for i in range(698):
        Herb_Target_P_item = Herb_Target_P[Herb_Target_P['Herb'] == i]
        Herb_Target_N_item = Herb_Target_N[Herb_Target_N['Herb'] == i]

        Herb_Target_P_item = Herb_Target_P_item.sort_values(by='Inferred evidence score', ascending=False)
        Herb_Target_N_item = Herb_Target_N_item.sort_values(by='Inferred evidence score', ascending=True)

        # rows = min(Herb_Target_P_item.shape[0], Herb_Target_N_item.shape[0])
        rows = 2
        Herb_Target_P_item = Herb_Target_P_item.iloc[:rows, :]
        Herb_Target_N_item = Herb_Target_N_item.iloc[:rows, :]

        positive_target = Herb_Target_P_item['Target'].tolist()
        negative_target = Herb_Target_N_item['Target'].tolist()
        positive_score = Herb_Target_P_item['Inferred evidence score'].tolist()
        negative_score = Herb_Target_N_item['Inferred evidence score'].tolist()

        data = pd.DataFrame({
            'herb': i,
            'positive_target': positive_target,
            'negative_target': negative_target,
            'positive_score': positive_score,
            'negative_score': negative_score
        })

        if not df_final.empty:
            df_final = pd.concat([df_final, data], axis=0)
        else:
            df_final = data

    df_final['positive_target'] = df_final['positive_target'].astype(int)
    df_final['negative_target'] = df_final['negative_target'].astype(int)
    df_final['positive_score'] = df_final['positive_score'].astype(float)
    df_final['negative_score'] = df_final['negative_score'].astype(float)
    df_final = df_final.sample(frac=1, random_state=random_seed).reset_index(drop=True)

    unique_herbs = df_final['herb'].unique()
    np.random.seed(random_seed)
    fold_mapping = {herb: np.random.randint(0, 5) for herb in unique_herbs}
    df_final['fold'] = df_final['herb'].map(fold_mapping)

    df_final.to_csv(f'../exp_dataset_0.250_0.200_min_2_select_2_Pv/seed={random_seed}.csv', index=None)